In [1]:
import os
import string
import re

import numpy as np

import joblib

from datetime import datetime as dt
import pandas as pd

import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

import matplotlib.pyplot as plt

In [2]:
from preprocessing_utils import preprocess_texts

2024-03-16 19:36:54 rhodes-f6eac metapub.config[266559] WARNING NCBI_API_KEY was not set.
2024-03-16 19:36:55.500714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 19:36:55.500766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 19:36:55.501614: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 19:36:55.508191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow 

In [3]:
DATA_PATH = '/data/02_training_data/'
MODEL_OUT_PATH = '/data/03_models/relation_classifier/'

In [4]:
try:
    train_df = pd.read_parquet(os.path.join(DATA_PATH,'temp_train_df.pq'))
    test_df = pd.read_parquet(os.path.join(DATA_PATH,'temp_test_df.pq'))
except FileNotFoundError as e:
    print(e)

In [5]:
print(train_df.shape)
print(train_df.type_of_regulation.value_counts())
train_df.head(2)

(72, 14)
type_of_regulation
antisense inhibitor of           51
regulates (molecular biology)    15
activator of                      6
Name: count, dtype: int64


,srna,srna_synonyms,type_of_regulation,target_gene,quote,quote_from,pmcid,wikidata,srna_name,text_prep,target_gene_id,gene_synonyms,srna_name_mentioned,gene_name_mentioned
0,sRNA encoded by micF,"stc, b4439, ECK2208, IS113, JWR0043, micF",antisense inhibitor of,outer membrane porin F b0929,we suggest that an increased expression of the...,/Article_Viewer%2Ehtml?pmcid=PMC213846&quote=w...,213846,http://www.wikidata.org/entity/Q50418533,micF,we suggest that an increased expression of the...,b0929,"cmlB, cry, ECK0920, nfxB, tolF, ompF",micF,ompF
1,sRNA encoded by ryhB,"b4451, ECK3426, IS176, JWR0219, psrA18, ryhB, ...",antisense inhibitor of,aconitate hydratase 1 b1276,The genes regulated by RyhB include some encod...,/Article_Viewer%2Ehtml?pmcid=PMC1251601&quote=...,1251601,http://www.wikidata.org/entity/Q50419794,ryhB,The genes regulated by RyhB include some encod...,b1276,"acn, ECK1271, acnA",RyhB,"acnA, acn"


In [6]:
print(test_df.shape)
print(test_df.type_of_regulation.value_counts())
test_df.head(2)

(27, 14)
type_of_regulation
antisense inhibitor of           17
regulates (molecular biology)     9
activator of                      1
Name: count, dtype: int64


,srna,srna_synonyms,type_of_regulation,target_gene,quote,quote_from,pmcid,wikidata,srna_name,text_prep,target_gene_id,gene_synonyms,srna_name_mentioned,gene_name_mentioned
0,sRNA encoded by omrB,"t59, b4445, ECK2834, JWR0249, omrB, PAIR2b, rygB",antisense inhibitor of,ferric dihyroxybenzoylserine outer membrane tr...,"Therefore, the first 10 nt of both OmrA and Om...",/Article_Viewer%2Ehtml?pmcid=PMC2588501&quote=...,2588501,http://www.wikidata.org/entity/Q50419398,omrB,Therefore the first 10 nt of both OmrA and Omr...,b2155,"cir, ECK2148, feuA, cirA",OmrB,"cir, cirA"
1,sRNA encoded by omrA,"b4444, ECK2833, JWR0248, omrA, PAIR2a, psrA12,...",antisense inhibitor of,outer membrane protease VII (outer membrane pr...,"Therefore, the first 10 nt of both OmrA and Om...",/Article_Viewer%2Ehtml?pmcid=PMC2588501&quote=...,2588501,http://www.wikidata.org/entity/Q50419343,omrA,Therefore the first 10 nt of both OmrA and Omr...,b0565,"ECK0557, ompT",OmrA,ompT


## Generate negative examples using GPT-4:
**Prompt used:**
> *"Generate 500 sentences that are structured and phrased as if extracted from scientific research articles or academic discussions in the fields of microbiology and molecular biology. These sentences should encompass a broad range of topics including but not limited to gene expression analysis, protein function and interaction, cellular processes, bacterial genetics, and responses to environmental stressors. Each sentence must be unique and present a specific scientific fact, hypothesis, observation, or methodological approach relevant to the aforementioned fields. Ensure the sentences reflect the precision, terminology, and tone typical of scholarly articles, including references to experimental techniques, genetic components, and theoretical frameworks without simulating references or citations. The objective is to create a dataset of negative examples that are contextually similar to discussions about bacterial sRNA regulation but are clearly distinguishable by their lack of specific references to sRNA-mediated regulatory mechanisms."*

In [7]:
from negative_examples import negative_examples
negative_examples = [preprocess_texts(ex) for ex in negative_examples]
len(negative_examples)

500

In [8]:
negative_examples_df = pd.DataFrame(negative_examples, columns=['text_prep'])
negative_examples_df['type_of_regulation'] = 'none'
print(negative_examples_df.shape)
negative_examples_df.head(3)

(500, 2)


,text_prep,type_of_regulation
0,The role of ATP synthase in mitochondrial ener...,none
1,Through chromatin immunoprecipitation assays t...,none
2,The study of bacterial conjugation reveals a h...,none


In [9]:
train_negative_df, test_negative_df =  train_test_split(negative_examples_df, test_size=0.3, random_state=42)

In [10]:
train_df = pd.concat([train_df, train_negative_df])[['text_prep', 'type_of_regulation']].drop_duplicates()
train_df = train_df.sample(frac=1).reset_index(drop=True)
print(train_df.shape, end='\n\n')
print(train_df.type_of_regulation.value_counts())
train_df.head(2)

(374, 2)

type_of_regulation
none                             350
antisense inhibitor of            14
regulates (molecular biology)      7
activator of                       3
Name: count, dtype: int64


,text_prep,type_of_regulation
0,Bacterial symbiosis with marine organisms impa...,none
1,The role of bacteria in atmospheric processes ...,none


In [11]:
test_df = pd.concat([test_df, test_negative_df])[['text_prep', 'type_of_regulation']].drop_duplicates()
test_df = test_df.sample(frac=1).reset_index(drop=True)
print(test_df.shape, end='\n\n')
print(test_df.type_of_regulation.value_counts())
test_df.head(2)

(161, 2)

type_of_regulation
none                             150
antisense inhibitor of             7
regulates (molecular biology)      3
activator of                       1
Name: count, dtype: int64


,text_prep,type_of_regulation
0,Bacterial strategies for the synthesis and mod...,none
1,The role of oxidative phosphorylation in ATP p...,none


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

nltk_stop_words = stopwords.words('english')

In [13]:
tfidf_vectorizer = TfidfVectorizer(stop_words=nltk_stop_words)

# This will store the class as key and its top 20 unique terms as values
unique_terms_per_class = {}

In [14]:
for label in train_df.type_of_regulation.unique():
    # Vectorize the current class texts
    tfidf_matrix = tfidf_vectorizer.fit_transform(train_df[train_df.type_of_regulation==label].text_prep.to_list())
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # Sum tfidf scores for each term across all documents in the class
    sum_tfidf = tfidf_matrix.sum(axis=0)
    tfidf_scores = [(feature_names[col], sum_tfidf[0, col]) for col in range(sum_tfidf.shape[1])]
    
    # Sort the terms by their score and select the top 20
    sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)[:50]
    
    # At this point, you have the top 20 terms for the current class
    # but they are not yet filtered for uniqueness across classes
    unique_terms_per_class[label] = [term for term, score in sorted_tfidf_scores]

    print(f'Top 20 terms for {label}:')
    print(unique_terms_per_class[label])
    print()

Top 20 terms for none:
['bacterial', 'gene', 'mechanisms', 'bacteria', 'microbial', 'role', 'potential', 'environmental', 'study', 'insights', 'understanding', 'production', 'strategies', 'including', 'processes', 'provides', 'application', 'cell', 'resistance', 'synthesis', 'regulation', 'stress', 'response', 'adaptation', 'roles', 'health', 'biosynthesis', 'exploration', 'protein', 'nutrient', 'development', 'energy', 'cycling', 'cellular', 'interactions', 'systems', 'offers', 'crucial', 'compounds', 'environments', 'degradation', 'investigation', 'survival', 'metabolic', 'highlights', 'nitrogen', 'metabolism', 'organic', 'involved', 'informs']

Top 20 terms for antisense inhibitor of:
['rna', 'gcvb', 'expression', 'genes', 'suggest', 'fur', 'targets', 'mrna', 'ryhb', 'srna', 'repressed', 'proteins', 'mutant', 'could', 'mrnas', 'results', 'regulation', 'rela', 'additional', 'omra', 'region', 'repression', 'previously', 'shown', 'regulate', 'also', 'regulated', 'reduced', 'soda', 'ana

#### Identify Terms within the top 30 TFIDF Terms per class, that are not in other classes

In [15]:
final_unique_terms_per_class = {}

for label, terms in unique_terms_per_class.items():
    other_classes_terms = set()
    for other_class, other_terms in unique_terms_per_class.items():
        if other_class != label:
            other_classes_terms.update(other_terms)
    
    # Filter terms that are not in any other class's top 20 list
    unique_terms = [term for term in terms if term not in other_classes_terms]
    final_unique_terms_per_class[label] = unique_terms
    
    print(f'Top terms for {label} that are not in the other classes:')
    print(final_unique_terms_per_class[label])
    print()

Top terms for none that are not in the other classes:
['bacterial', 'gene', 'mechanisms', 'bacteria', 'microbial', 'potential', 'environmental', 'study', 'insights', 'understanding', 'production', 'strategies', 'including', 'processes', 'provides', 'application', 'cell', 'resistance', 'synthesis', 'stress', 'response', 'adaptation', 'roles', 'health', 'biosynthesis', 'exploration', 'protein', 'nutrient', 'development', 'energy', 'cycling', 'cellular', 'systems', 'offers', 'crucial', 'compounds', 'environments', 'degradation', 'investigation', 'survival', 'metabolic', 'highlights', 'nitrogen', 'metabolism', 'organic', 'informs']

Top terms for antisense inhibitor of that are not in the other classes:
['suggest', 'fur', 'targets', 'ryhb', 'srna', 'proteins', 'mutant', 'could', 'mrnas', 'rela', 'additional', 'region', 'repression', 'shown', 'regulate', 'regulated', 'reduced', 'soda', 'analysis', 'dppa', 'gfp', 'oppa', 'encoding', 'transport', '124', '161', 'cyca', 'pairing', 'required', '

In [16]:
train_df = train_df[['text_prep', 'type_of_regulation',]].drop_duplicates().reset_index(drop=True)
test_df = test_df[['text_prep', 'type_of_regulation']].drop_duplicates().reset_index(drop=True)

print(f'Train size: {train_df.shape}')
print(f'Test size:{test_df.shape}')

Train size: (374, 2)
Test size:(161, 2)


In [17]:
test_df.type_of_regulation.value_counts()

type_of_regulation
none                             150
antisense inhibitor of             7
regulates (molecular biology)      3
activator of                       1
Name: count, dtype: int64

### Calculate POS Tags

In [18]:
import spacy
pos_tagger = spacy.load("en_core_sci_sm")

/home/bernards/sRNA_extraction/srna_venv/lib/python3.10/site-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [249]:
class POSTagsTfidfVectorizer(BaseEstimator, TransformerMixin):
    """
    A custom transformer that converts text to Part-Of-Speech (POS) tags using spaCy
    and then applies TF-IDF vectorization to these tags.
    """
    
    def __init__(self, model_name: str = "en_core_sci_sm"):
        """
        Initializes the transformer with a spaCy language model for POS tagging
        and a TfidfVectorizer for vectorizing the POS tags.
        """
        self.model_name = model_name
        self.pos_tagger = spacy.load(model_name)
        self.tfidf_vectorizer = TfidfVectorizer()
    
    def fit(self, X, y=None):
        """
        Fits the TfidfVectorizer to the POS tags generated from the input text.

        Parameters:
            X (iterable of str): The input text data to fit.
            y (ignored): Not used.

        Returns:
            self (POSTagsTfidfVectorizer): The fitted transformer.
        """
        pos_tags = self._texts_to_pos_tags(X)
        self.tfidf_vectorizer.fit(pos_tags)
        return self
    
    def transform(self, X, y=None):
        """
        Transforms the input text to a TF-IDF representation of its POS tags.

        Parameters:
            X (iterable of str): The input text data to transform.

        Returns:
            transformed (sparse matrix): The TF-IDF representation of the POS tags.
        """
        pos_tags = self._texts_to_pos_tags(X)
        transformed = self.tfidf_vectorizer.transform(pos_tags)
        return transformed
    
    def _texts_to_pos_tags(self, texts):
        """
        Helper method to convert a list of texts to a list of space-separated POS tags strings.

        Parameters:
            texts (iterable of str): The input text data.

        Returns:
            pos_tags_list (List[str]): A list of space-separated POS tags strings.
        """
        pos_tags_list = [' '.join([token.pos_ for token in self.pos_tagger(text)]) for text in texts]
        return pos_tags_list

### Create RegEx Patterns

In [250]:
class RegexFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.patterns = {
            'activator_of': r"activat(es?|ion|or|ing)",
            'regulation': r"re(gulat(e|es|ed|ing|ion)|press(ed|ing)?)",
            'antisense_inhibitor': r"(antisense|inhibit(s|ed|ing|ion))"
            }
    
    def fit(self, X, y=None):
        return self  # Nothing to fit, so just return self
    
    def transform(self, X, y=None):
        # Initialize an array for the extracted features
        features = np.zeros((len(X), len(self.patterns)), dtype=int)
        
        # Iterate over the rows in the input data
        for i, text in enumerate(X):
            for j, (label, pattern) in enumerate(self.patterns.items()):
                if re.search(pattern, text, re.IGNORECASE):
                    features[i, j] = 1
        
        return features

In [251]:
pos_tags_tfidf_vectorizer = POSTagsTfidfVectorizer()
regex_feature_extractor = RegexFeatureExtractor()


## Encode Labels

In [15]:
label_encoder = LabelEncoder()
train_df['labels'] = label_encoder.fit_transform(train_df['type_of_regulation'])
test_df['labels'] = label_encoder.fit_transform(test_df['type_of_regulation'])
y_train = train_df['labels']
y_test = test_df['labels']


In [ ]:
label_encoder.classes_

In [ ]:
X_train = train_df[['text_prep']]

X_test = test_df[['text_prep']]

In [ ]:
train_df[(train_df.text_prep.isin(test_df.text_prep))&(train_df.type_of_regulation!="none")]

# Model Exploration
## Set up BioBERT Sentence Tokenizer

## Create Preprocesser for Pipeline

In [ ]:
# Base transformers
text_tfidf = TfidfVectorizer()
text_tfidf_remove_stopwords = TfidfVectorizer(stop_words=nltk_stop_words)
pos_tfidf = TfidfVectorizer()

In [ ]:
feature_combinations = [
    # Only Text TFIDF
    {'name': 'only_text_tfidf', 
     'transformers': [
         ('text_tfidf', text_tfidf, 'text_prep')
     ]
    },
    # Text TFIDF + POS Tags TFIDF
    {'name': 'text_tfidf+pos_tfidf', 
     'transformers': [
         ('text_tfidf', text_tfidf, 'text_prep'), 
         ('pos_tfidf', pos_tags_tfidf_vectorizer, 'text_prep')
         ]
    },
    # Text TFIDF + RegEx Features
    {'name': 'text_tf_idf+regex', 
     'transformers': [
         ('text_tfidf', text_tfidf, 'text_prep'), 
         ('regex_features', regex_feature_extractor, 'text_prep')
     ]
    },
    # Text TFIDF + POS Tags TFIDF + RegEx Features
    {'name': 'text_tf_idf+pos_tfidf+regex', 
     'transformers': [
         ('text_tfidf', text_tfidf, 'text_prep'), 
         ('pos_tfidf', pos_tags_tfidf_vectorizer, 'text_prep'), 
         ('regex_features', regex_feature_extractor, 'text_prep')
     ]
    },
    
    # Only Text TFIDF without stopwords
    {'name': 'only_text_tfidf_remove_stopwords', 
     'transformers': [
         ('text_tfidf_remove_stopwords', text_tfidf_remove_stopwords, 'text_prep')
     ]
    },
    # Text TFIDF without stopwords + POS Tags TFIDF 
    {'name': 'text_tfidf_remove_stopwords+pos_tfidf', 
     'transformers': [
         ('text_tfidf_remove_stopwords', text_tfidf_remove_stopwords, 'text_prep'), 
         ('pos_tfidf', pos_tags_tfidf_vectorizer, 'text_prep')
     ]
    },
    # Text TFIDF without stopwords + RegEx Features    
    {'name': 'text_tfidf_remove_stopwords+regex', 
     'transformers': [
         ('text_tfidf_remove_stopwords', text_tfidf_remove_stopwords, 'text_prep'), 
         ('regex_features', regex_feature_extractor, 'text_prep')
     ]
    },
    # Text TFIDF without stopwords + POS Tags TFIDF + RegEx Features    
    {'name': 'text_tf_idf_remove_stopwords+regex+pos_tfidf', 
    'transformers': [
        ('text_tfidf_remove_stopwords', text_tfidf_remove_stopwords, 'text_prep'),
        ('pos_tfidf', pos_tags_tfidf_vectorizer, 'text_prep'), 
        ('regex_features', regex_feature_extractor, 'text_prep')
        ]
    },
]

## Define classifiers to compare

In [ ]:
classifiers = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(kernel='linear'),
    'SVM_sigmoid': SVC(kernel='sigmoid'),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'MultinomialNB': MultinomialNB()    
}

## Compare classifier performances
### Run each classifier/preprocessing combination to find the optimal pipeline

In [ ]:
results = []

for combo in feature_combinations:
    preprocessor = ColumnTransformer(transformers=combo['transformers'])
    for name, classifier in classifiers.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', classifier)])
        
        # Train
        pipeline.fit(X_train, y_train)
        
        # Evaluate
        y_pred = pipeline.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for imbalanced classes
        
        results.append((name, combo['name'], f1))

# Sort the results based on F1 score in descending order and select the top 3
top_3_models = sorted(results, key=lambda x: x[2], reverse=True)[:3]

print("\nTop 3 Models based on F1 Score:")
for model in top_3_models:
    print(f"Model: {model[0]} with {model[1]}: F1 Score = {model[2]}")

## Parameter Tuning for Best Model
### Select optimal feature combination from previous experiment

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text_tfidf_remove_stopwords', TfidfVectorizer(stop_words=nltk_stop_words), 'text_prep'),
        ('pos_tfidf', pos_tags_tfidf_vectorizer, 'text_prep'),
        ('regex_features', regex_feature_extractor, 'text_prep')
    ]
)

### Set up GridSearch for Parameter Tuning

In [ ]:
param_grid = {
    'classifier__C': [0.1, 0.5, 1.0, 1.5, 2, 2.5],
    'classifier__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'classifier__gamma': ['scale', 'auto', 0.1, 1, 10],
    'classifier__degree': [2, 3, 4]
}

In [ ]:
# Initialize the classifier pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])
# Define the F1 scorer
f1_scorer = make_scorer(f1_score, average='weighted')

# Setup the grid search
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=f1_scorer, cv=5, verbose=2, n_jobs=8)

## Perform GridSearch

In [ ]:
# Assuming X_train and y_train are your features and labels for training
grid_search.fit(X_train, y_train)


### Show results for best model

In [ ]:
print("Best parameters found: ", grid_search.best_params_)
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test) 
print(classification_report(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='weighted')
print(f1)

In [ ]:
test_df[test_df.type_of_regulation=='activator of'].text_prep.iloc[0]

## Train the final model

In [ ]:
final_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(
        C=grid_search.best_params_['classifier__C'],
        kernel=grid_search.best_params_['classifier__kernel'],
        gamma=grid_search.best_params_['classifier__gamma'],
        degree=grid_search.best_params_['classifier__degree'],
        random_state=42))
])

In [ ]:
final_model.fit(X_train, y_train)

In [ ]:
y_pred = final_model.predict(X_test)
print(classification_report(label_encoder.inverse_transform(y_test), label_encoder.inverse_transform(y_pred)))
f1 = f1_score(y_test, y_pred, average='weighted')
print(f1)

In [ ]:
label_names = [label.split()[0] for label in label_encoder.classes_]
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names)

# Display as a heatmap
disp.plot(cmap=plt.cm.Blues, )
plt.title('Relation Classification Confusion Matrix')
# plt.xticks(rotation=30)
plt.show()

## Write best model to disk

In [ ]:
joblib.dump(final_model, os.path.join(MODEL_OUT_PATH, 'relation_classifier.joblib'))

In [ ]:
joblib.dump(label_encoder, os.path.join(MODEL_OUT_PATH, 'label_encoder.joblib'))